# Simple NN

Investigate a simple feedforward NN. Look at 1 layer then multi layer?

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler

from os.path import join

import sys
sys.path.append('../')
import utils

In [2]:
assert torch.cuda.is_available(), 'CUDA is not available.'

## Load Data

In [3]:
fname = join('/home/squirt/Documents/data/weather_data/', 'all_data.h5')

Set batchsize and partition. Create Dataloaders from h5 data. Use utils function

In [4]:
batch_size = 2048 
split = 0.5
t_dl, v_dl = utils.get_dataloaders(fname, batch_size, split)

## Define Model

Simple FF net. Flatten input so $(3*2*2 + 70*2*2*2)=852$. And flatten output $(70*2*2*2)=560$.

In [5]:
class simpleNN(nn.Module):
    def __init__(self):
        super(simpleNN, self).__init__()
        self.fc1 = nn.Linear(852,4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 560)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model = simpleNN()
model = model.double().cuda()

## Define Training Loop 

In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

Define training loop. Does one pass over data then returns average loss
Inputs:
    - model (nn.Module): Our Net that we want to optimize
    - dl (torch.utils.data.DataLoader): Train dataloader
    - optim (torch.optim): Optimizer to train
    - loss (nn.Module): Loss function (can be nn.Module or nn.Function)
Returns:
    - Training loss on one pass


In [8]:

def train(model:nn.Module, dl:torch.utils.data.DataLoader, optim:torch.optim, loss:nn.Module) -> float:
    model.train()
    total_loss = .0
    scaler = GradScaler()

    for _, (l, x, y) in enumerate(dl):
        l = l.cuda()
        x = x.cuda()
        y = y.cuda()

        # Flatten and combine
        l = l.view(-1, 3*2*2)
        x = x.view(-1, 70*3*2*2)
        x = torch.cat((l, x), 1)

        y = y.view(-1, 70*2*2*2)

        # Forward pass
        with autocast():
            y_pred = model(x)
            l = loss(y_pred, y)
            total_loss += l.item()

        # Preform backpass
        scaler.scale(l).backward()
        scaler.step(optim)
        scaler.update()
    
    return total_loss / len(dl)

Define eval loss loop. Does one pass over evaluation data and returns the average loss
Inputs:
    - model (nn.Module): Network that we are training
    - dl (torch.utils.data.DataLoader): Eval dataloader

In [9]:
def eval(model:nn.Module, dl:torch.utils.data.DataLoader, loss:nn.Module) -> float:
    model.eval()
    total_loss = .0

    for _, (l, x, y) in enumerate(dl):
        l = l.cuda()
        x = x.cuda()
        y = y.cuda()

        # Flatten and combine
        l = l.view(-1, 3*2*2)
        x = x.view(-1, 70*3*2*2)
        x = torch.cat((l, x), 1)

        y = y.view(-1, 70*2*2*2)

        # Forward pass
        with autocast():
            y_pred = model(x)
            l = loss(y_pred, y)
            total_loss += l.item()

    return total_loss / len(dl)

## Training

In [10]:
train_loss = nn.MSELoss()

In [11]:
for i in range(26):
    train(model, t_dl, optimizer, train_loss)
    l = eval(model, v_dl, train_loss)
    print(f'Epoch {i+1} - Eval Loss: {l}')

Epoch 1 - Eval Loss: 482938.94335975574
Epoch 2 - Eval Loss: 11401.991708544698
Epoch 3 - Eval Loss: 1104.8450253593107
Epoch 4 - Eval Loss: 264.06146701162027
Epoch 5 - Eval Loss: 60.89480827992425
Epoch 6 - Eval Loss: 16.302699812532502
Epoch 7 - Eval Loss: 1.4471300351442102
Epoch 8 - Eval Loss: 0.7985396964771315
Epoch 9 - Eval Loss: 0.6212278543945493
Epoch 10 - Eval Loss: 0.5706674427013242
Epoch 11 - Eval Loss: 0.5046172889658046
Epoch 12 - Eval Loss: 0.4302757526406055
Epoch 13 - Eval Loss: 0.3602562280067849
Epoch 14 - Eval Loss: 0.3060196015674116
Epoch 15 - Eval Loss: 0.27500417940345595
Epoch 16 - Eval Loss: 0.26617113650604446
Epoch 17 - Eval Loss: 0.26556703719904595
Epoch 18 - Eval Loss: 0.26451142933533156
Epoch 19 - Eval Loss: 0.2627033008641057
Epoch 20 - Eval Loss: 0.26006561942436646
Epoch 21 - Eval Loss: 0.25896692858648296
Epoch 22 - Eval Loss: 0.2582646581252509
Epoch 23 - Eval Loss: 0.25803565253025756
Epoch 24 - Eval Loss: 0.2583166518725482
Epoch 25 - Eval Los

In [14]:
print(len(t_dl), len(v_dl))
print(len(t_dl.dataset), len(v_dl.dataset))

5 5
9600 9600
